### Entranamiento / Evaluación

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 💪 Training

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [3]:
proc_data_path='../data/processed/road_traffic_accidents_dataset_proc.csv'
train_data = pd.read_csv(proc_data_path)


In [4]:
X = train_data.drop('accident_severity', axis=1)
y = train_data['accident_severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)


In [6]:
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))


Model accuracy score with 10 decision-trees : 0.8754


In [7]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       1.00      0.04      0.09        67
           2       0.87      1.00      0.93       503

    accuracy                           0.88       578
   macro avg       0.62      0.35      0.34       578
weighted avg       0.88      0.88      0.82       578



c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [8]:

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, cv=5, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

print('Average cross-validation score: {}'.format(scores.mean()))


Cross-validation scores:[0.85640138 0.85294118 0.85294118 0.85294118 0.85961872]
Average cross-validation score: 0.8549687261998284


In [9]:
import joblib

# Save the model as a pickle file
filename = '../models/rf_model.pkl'
joblib.dump(rf, filename)

# Load the model from the file
rf_model = joblib.load(filename)

# Use the loaded model to make predictions
y_pred = rf_model.predict(X_test)

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

print('Classification report : ')

print(classification_report(y_test, y_pred))

print('Cross-validation scores :')

scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')

print(scores)

print('Average cross-validation score : {}'.format(scores.mean()))



Model accuracy score with 10 decision-trees : 0.8754
Classification report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       1.00      0.04      0.09        67
           2       0.87      1.00      0.93       503

    accuracy                           0.88       578
   macro avg       0.62      0.35      0.34       578
weighted avg       0.88      0.88      0.82       578

Cross-validation scores :


c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Diego Gerwig\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

[0.85640138 0.85294118 0.85294118 0.85294118 0.85961872]
Average cross-validation score : 0.8549687261998284


#### 🎯 Predict